<h1 align="center"><font color="yellow">LangChain 0: GPT-3 vs LLMs (código aberto)</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Neste script seguiremos os tutoriais de [James Briggs](https://www.pinecone.io/learn/langchain-intro/) 😎.

# Contextualizando

`LangChain` é um Framework popular que permite aos usuários criar rapidamente aplicativos e pipelines em torno de `Large Language Models` (LLMs). Ele se integra diretamente aos modelos `GPT-3` e `GPT-3.5` da `OpenAI` e às alternativas de código aberto do `Hugging Face`, como os modelos `flan-t5 do Google`. Ele pode ser usado para `chatbots`, `perguntas-respostas generativas` (GQA), `resumos` e muito mais. A ideia central da biblioteca é que podemos `"encadear"` (chain) diferentes componentes para criar casos de uso mais avançados em torno de LLMs. As cadeias podem consistir em vários componentes de vários módulos. 

# Introdução

Começamos citando e descrevendo as quatro componentes básicas:

* `Prompt templates:` Os templates de prompt são, bem, modelos para diferentes tipos de prompts. Como modelos de estilo `"chatbot"`, respostas a perguntas `ELI5`, etc.

* `LLMs:` Modelos de Linguagem Grandes como `GPT-3`, `BLOOM`, etc.

* `Agentes:` os agentes usam LLMs para decidir quais ações devem ser executadas, ferramentas como pesquisa na Web ou calculadoras podem ser usadas e tudo empacotado em um loop lógico de operações.

* `Memória:` memória de curto prazo, memória de longo prazo.

In [ ]:
#%pip install -qU langchain

In [1]:
# Isto é quando usas o arquivo .env:
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 
Eddy_API_KEY_HuggingFace = os.environ["HUGGINGFACEHUB_API_TOKEN"]


Carregando a minha chave Key:  True


# Usando LLMs em LangChain

<font color="orange">O `LangChain` oferece suporte a vários provedores de LLM, como `Hugging Face` e `OpenAI`.

Vamos começar nossa exploração do `LangChain` aprendendo como usar algumas dessas diferentes integrações LLM.</font>

# <font color="red">Hugging Face</font>

In [2]:
%pip install -qU huggingface_hub

Note: you may need to restart the kernel to use updated packages.


Podemos então gerar texto usando um modelo HF Hub (usaremos `google/flan-t5-x1`) usando a `API de inferência` incorporada ao `Hugging Face Hub`.

(A API de inferência padrão não usa hardware especializado e, portanto, pode ser lenta e não pode executar modelos maiores como `bigscience/bloom-560m` ou `google/flan-t5-xxl`)

In [40]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain


# Inicializar HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":1e-10}
)

# Crie um modelo de prompt para responder a perguntas simples
template = """


Question: {question}
Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])


llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5
)

question = "O que é física quântica?"

print(llm_chain.run(question))


quantum mechanics


<font color="orange">Se quisermos fazer várias perguntas, podemos passar uma lista de objetos de dicionário, onde os dicionários devem conter a variável de entrada definida em nosso modelo de `prompt` (`"question"`) que é mapeada para a pergunta que gostaríamos de fazer.</font>

In [23]:

qs = [
    {'question': "Qual time da NFL venceu o Super Bowl na temporada de 2010?"},
    {'question': "Se eu tenho 6 pés e 4 polegadas, qual a minha altura em centímetros?"},
    {'question': "Quem foi a 12ª pessoa na lua?"},
    {'question': "Quantos olhos tem uma lâmina de grama?"}
]
res = llm_chain.generate(qs)
res



LLMResult(generations=[[Generation(text='san francisco 49ers', generation_info=None)], [Generation(text='240', generation_info=None)], [Generation(text='jupiter', generation_info=None)], [Generation(text='2', generation_info=None)]], llm_output=None)

<font color="orange">É um LLM, então podemos tentar alimentar todas as perguntas de uma só vez:</font>

In [24]:
multi_template = """Responda às seguintes perguntas, uma de cada vez.

Questions:
{questions}

Answers:
"""

long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)


llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)


qs_str = (
    "Qual time da NFL venceu o Super Bowl na temporada de 2010?\n" +
    "Se eu tenho 6 pés e 4 polegadas, qual a minha altura em centímetros?\n" +
    "Quem foi a 12ª pessoa na lua?" +
    "Quantos olhos tem uma lâmina de grama?"
)


print(llm_chain.run(qs_str))


Qual tempo da NFL venceu o Super Bowl na tempo


<font color="orange">Mas com este modelo não funciona muito bem, veremos que esta abordagem funciona melhor com `modelos diferentes` em breve.</font>

# <font color="red">OpenAI</font> 

In [ ]:
#%pip install -qU openai

<font color="orange">Então, decidimos qual modelo gostaríamos de usar, existem várias opções, mas iremos com `text-davinci-003`:</font>

In [11]:
from langchain.llms import OpenAI

davinci = OpenAI(model_name='text-davinci-003')


<font color="orange">Como alternativa, se estiver usando o `Azure OpenAI`, fazemos:</font>

```
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name="your-azure-deployment", 
    model_name="text-davinci-003"
)
```


<font color="orange">Usaremos o mesmo modelo de `prompt` de `pergunta-resposta simples` de antes com o exemplo de `Hugging Face`. A única mudança é que agora passamos nosso `OpenAI LLM davinci`:</font>

In [12]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(llm_chain.run(question))

 Os New Orleans Saints venceram o Super Bowl na temporada de 2010.


O mesmo funciona novamente para `várias perguntas` usando `generate`:

In [16]:
qs = [
    {'question': "Qual time da NFL venceu o Super Bowl na temporada de 2010?"},
    {'question': "Se eu tenho 6 pés e 4 polegadas, qual a minha altura em centímetros?"},
    {'question': "Quem foi a 12ª pessoa na lua?"},
    {'question': "Quantos olhos tem uma lâmina de grama?"}
]

llm_chain.generate(qs)


LLMResult(generations=[[Generation(text=' Os New Orleans Saints venceram o Super Bowl na temporada de 2010.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' 193.04 cm', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' O astronauta Eugene Cernan foi a 12ª pessoa a pisar na Lua. Ele foi o último homem a pisar na Lua durante a missão Apollo 17, em dezembro de 1972.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' Uma lâmina de grama não tem olhos.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 192, 'completion_tokens': 91, 'prompt_tokens': 101}, 'model_name': 'text-davinci-003'})

<font color="orange">Observe que o `formato abaixo` não alimenta as perguntas de forma `iterativa`, mas todas em um bloco.</font>

In [20]:
qs = [
    "Qual time da NFL venceu o Super Bowl na temporada de 2010?",
    "Se eu tenho 6 pés e 4 polegadas, qual a minha altura em centímetros?",
    "Quem foi a 12ª pessoa na lua?",
    "Quantos olhos tem uma lâmina de grama?"
     ]

print(llm_chain.run(qs))



1. O New Orleans Saints foi o vencedor do Super Bowl na temporada de 2010.
2. Sua altura em centímetros é 193.04.
3. A 12ª pessoa na lua foi o astronauta Harrison Schmitt em 1972.
4. Uma lâmina de grama não tem olhos.


<font color="orange">Agora podemos tentar responder a todas as perguntas de uma só vez, como mencionado, `LLMs` mais poderosos como `text-davinci-003` terão maior probabilidade de lidar com essas consultas mais complexas.</font>

In [22]:
multi_template = """Responda às seguintes perguntas, uma de cada vez.

Questions:
{questions}

Answers:
"""

long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)


qs_str = (
    "Qual time da NFL venceu o Super Bowl na temporada de 2010?\n" +
    "Se eu tenho 6 pés e 4 polegadas, qual a minha altura em centímetros?\n" +
    "Quem foi a 12ª pessoa na lua?" +
    "Quantos olhos tem uma lâmina de grama?"
)


print(llm_chain.run(qs_str))

Qual time da NFL venceu o Super Bowl na temporada de 2010? Os Green Bay Packers.

Se eu tenho 6 pés e 4 polegadas, qual a minha altura em centímetros? 193,04 centímetros.

Quem foi a 12ª pessoa na lua? O americano Harrison Schmitt.

Quantos olhos tem uma lâmina de grama? Uma lâmina de grama não tem olhos.
